In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [0]:
import os
os.chdir("drive") 

In [0]:
import torch
import torch.nn as nn
import numpy as np
import torch.nn.functional as F

from torch.nn.utils import clip_grad_norm_

from corpus import Corpus
#from lstm import Lstm
from lstm_matrix import Lstm


In [2]:
# use gpu if it is possible
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)


cuda


In [0]:
# Hyperparameter

embed_size = 128     # Vector length to represent a word
hidden_size = 1024   # hidden size of LSTM
num_layers = 1      # only one layer
num_epochs1 = 2      # num of iterations with lr = lr
num_epochs2 = 4   # num of iterations with lr = lr*0.1 # 可以改成3
num_epochs3 = 3     # num of iterations with lr = lr*0.05
num_epochs4= 2     # num of iterations with lr = lr*0.02

num_epochs = num_epochs1 +num_epochs2 +num_epochs3 + num_epochs4
batch_size = 20     # Number of sentences in one batch (if too small: biger error due to randomness)
seq_length = 25     # Number of  words in one sentence
learning_rate = 0.01 #  

In [4]:
#Load data 
corpus = Corpus()
ids = corpus.get_data('train.csv', batch_size,seq_length)
vocab_size = len(corpus.dictionary) 
num_batches = ids.size(1) // seq_length 
print(vocab_size)

119598


In [0]:
train_model = Lstm(batch_size,vocab_size, embed_size, hidden_size, num_layers).to(device)

# choose loss function: 
criterion = nn.CrossEntropyLoss()  
#choose optimizer function 
optimizer = torch.optim.Adam(train_model.parameters(), lr=learning_rate)
optimizer2 = torch.optim.Adam(train_model.parameters(), lr=learning_rate * 0.1)
optimizer3 = torch.optim.Adam(train_model.parameters(), lr=learning_rate * 0.05)
optimizer4 = torch.optim.Adam(train_model.parameters(), lr=learning_rate * 0.02)



In [6]:
# Backpropagation process "truncated" (do not copy gradient)
def detach(states):
    return [state.detach() for state in states] 

# training...
 
for epoch in range(num_epochs):
    h_pre = torch.zeros(batch_size,hidden_size).to(device)
    c_pre = torch.zeros(batch_size,hidden_size).to(device)     
    loss = 0.0
    
    if (epoch ==num_epochs1+num_epochs2 +num_epochs3):
        torch.save(train_model.state_dict(), 'lstm_epoc_3.ckpt')
        optimizer = optimizer4
      
    if (epoch ==num_epochs1+num_epochs2):
        torch.save(train_model.state_dict(), 'lstm_epoc_2.ckpt')
        optimizer = optimizer3
        
    elif (epoch ==num_epochs1):
        torch.save(train_model.state_dict(), 'lstm_epoc_1.ckpt')
        optimizer = optimizer2
        
    for i in range(0, ids.size(1)-1 , seq_length):##  总的列数减1，间隔一句话的长度,0,25,50....
        loss = 0.0
        inputs = ids[:,i:i+seq_length-1].to(device)  #  [batch_size,seq_length-1]:[20,24]
        inputs = inputs.t() # [24,20]
        targets = ids[:,(i+1):i+seq_length].to(device) # Output is delayed by one word after the input
        outputs,(h_t,c_t) = train_model(inputs,h_pre,c_pre)
        loss = criterion(outputs,targets.reshape(-1))  
        
        # the 3 key steps
        train_model.zero_grad()
        loss.backward( )
        clip_grad_norm_(train_model.parameters(),0.5)  # max norm of the gradients to avoid gradient explode
        optimizer.step()
        
        step = (i+1) // seq_length
        if step % 200 == 0:
            print ('the current epoch [{}/{}], step_num in this epoch[{}/{}], Loss: {:.4f},Perplexity: {:5.2f}'
                   .format(epoch+1, num_epochs, step, num_batches, loss.item(), np.exp(loss.item())))
            
      
      


the current epoch [1/11], step_num in this epoch[0/2644], Loss: 11.6942,Perplexity: 119879.97
the current epoch [1/11], step_num in this epoch[200/2644], Loss: 7.2569,Perplexity: 1417.85
the current epoch [1/11], step_num in this epoch[400/2644], Loss: 6.8829,Perplexity: 975.44
the current epoch [1/11], step_num in this epoch[600/2644], Loss: 7.4077,Perplexity: 1648.65
the current epoch [1/11], step_num in this epoch[800/2644], Loss: 7.2093,Perplexity: 1351.90
the current epoch [1/11], step_num in this epoch[1000/2644], Loss: 7.0277,Perplexity: 1127.46
the current epoch [1/11], step_num in this epoch[1200/2644], Loss: 6.5548,Perplexity: 702.58
the current epoch [1/11], step_num in this epoch[1400/2644], Loss: 6.4832,Perplexity: 654.05
the current epoch [1/11], step_num in this epoch[1600/2644], Loss: 6.9085,Perplexity: 1000.71
the current epoch [1/11], step_num in this epoch[1800/2644], Loss: 7.2468,Perplexity: 1403.64
the current epoch [1/11], step_num in this epoch[2000/2644], Loss: 

In [0]:
torch.save({"model_state_dict" : train_model.state_dict(),
            "optimizer_state_dict" : optimizer.state_dict()},
            'lstm_epoc_save.ckpt')


In [10]:
checkpoint = torch.load('lstm_epoc_save.ckpt')
lstm_test_model = Lstm(batch_size,vocab_size, embed_size, hidden_size, num_layers).to(device)
lstm_test_model.load_state_dict(checkpoint["model_state_dict"])
optimizer.load_state_dict(checkpoint["optimizer_state_dict"])
print(lstm_test_model)

Lstm(
  (embed): Embedding(119598, 128)
  (linear): Linear(in_features=1024, out_features=119598, bias=True)
)


In [13]:
with torch.no_grad():
    num_samples = 1000
    with open('sample_0609.txt', 'w') as f:
         # initialize
        (h,c) = (torch.zeros(num_layers, 1, hidden_size).to(device),
                 torch.zeros(num_layers, 1, hidden_size).to(device))

        # Randomly select a word as input
        prob = torch.ones(vocab_size)
        input = torch.multinomial(prob, num_samples=1).unsqueeze(1).to(device)

        for i in range(num_samples):
            output, (h,c)  = lstm_test_model(input, h,c) 
            # predictions
            prob = output.exp()
            word_id = torch.multinomial(prob, num_samples=1).item()

            # Fill in preditions(input data for the next prediticon)
            input.fill_(word_id)

            # write the predictions 
            word = corpus.dictionary.idx2word[word_id]
            word = '\n' if word == '<eos>' else word + ' '
            f.write(word)

            if (i+1) % 100 == 0:
                print('generate [{}/{}] words，save in {}'.format(i+1, num_samples, 'sample_0609.txt'))
                


generate [100/1000] words，save in sample_0609.txt
generate [200/1000] words，save in sample_0609.txt
generate [300/1000] words，save in sample_0609.txt
generate [400/1000] words，save in sample_0609.txt
generate [500/1000] words，save in sample_0609.txt
generate [600/1000] words，save in sample_0609.txt
generate [700/1000] words，save in sample_0609.txt
generate [800/1000] words，save in sample_0609.txt
generate [900/1000] words，save in sample_0609.txt
generate [1000/1000] words，save in sample_0609.txt


writer: @ hebaodan